# Examples

Here we will showcase different types of problems that the package can solve. The examples can also be downloaded as a Jupyter notebook.

We begin by importing the functions that we will need:

In [1]:
from causalinflation import InflationProblem, InflationSDP
from examples_utils import bisection, P_W_array, P_2PR_array, P_PRbox_array

## Feasibility problems and extraction of certificates

### Example 1: Infeasibility of the W distribution in the quantum triangle scenario 

Consider determining if the following distribution, the so-called "W distribution" (due to its similarity to the [W state](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.62.062314)), is compatible with the triangle scenario:

$$ P_{A B C}=\frac{[100]+[010]+[001]}{3}, \quad \text {i.e.,} \quad P_{A B C}(a b c)= \begin{cases}\frac{1}{3} & \text { if } a+b+c=1, \\ 0 & \text { otherwise. }\end{cases} $$

It is known that it is [incompatible with the classical triangle scenario](https://www.degruyter.com/document/doi/10.1515/jci-2017-0020/html), however with quantum inflation, once can also show that it is incompatible with the quantum triangle scenario, depicted in the following figure:

<center> <img src="./figures/quantum_triangle.PNG" alt="drawing" width="250"/> </center>

To show this, we can generate the semidefinite relaxation of NPA level 2 corresponding to a second order quantum inflation:

In [2]:
qtriangle = InflationProblem(dag={"rho_AB": ["A", "B"],
                                  "rho_BC": ["B", "C"],
                                  "rho_AC": ["A", "C"]}, 
                             outcomes_per_party=[2, 2, 2],
                             settings_per_party=[1, 1, 1],
                             inflation_level_per_source=[2, 2, 2])
sdprelax = InflationSDP(qtriangle)
sdprelax.generate_relaxation('npa2')

With the `set_distribution` method we set the entries of the moment matrix that depend on the probability distribution, and we attempt to solve the program:

In [4]:
sdprelax.set_distribution(P_W_array)
sdprelax.solve()
sdprelax.status

'infeasible'

The problem status is reported as infeasible, therefore this serves as a proof that the W distribution is incompatible with the quantum triangle scenario. 

#### Certificate extraction

We can furthermore recover a certificate of infeasibility as a polynomial inequality in the probabilities, $\text{Poly}(p(abc|xyz) \geq 0$. There are built-in methods to extract the symbolic form of $\text{Poly}(p(abc|xyz))$:

In [5]:
sdprelax.certificate_as_probs(clean=True)

0.46*p(000|000)*p_{A}(0|0) + 0.456*p(000|000)*p_{B}(0|0) + 0.457*p(000|000)*p_{C}(0|0) + 0.616*p(000|000) + 0.232*p_{AB}(00|00)**2 + 0.103*p_{AB}(00|00)*p_{AC}(00|00) + 0.046*p_{AB}(00|00)*p_{A}(0|0) + 0.096*p_{AB}(00|00)*p_{BC}(00|00) + 0.041*p_{AB}(00|00)*p_{B}(0|0) - 0.365*p_{AB}(00|00)*p_{C}(0|0) + 0.131*p_{AB}(00|00) + 0.231*p_{AC}(00|00)**2 + 0.046*p_{AC}(00|00)*p_{A}(0|0) + 0.095*p_{AC}(00|00)*p_{BC}(00|00) - 0.363*p_{AC}(00|00)*p_{B}(0|0) + 0.045*p_{AC}(00|00)*p_{C}(0|0) + 0.13*p_{AC}(00|00) - 0.454*p_{A}(0|0)**2 - 0.365*p_{A}(0|0)*p_{BC}(00|00) + 1.0*p_{A}(0|0)*p_{B}(0|0)*p_{C}(0|0) - 0.732*p_{A}(0|0)*p_{B}(0|0) - 0.731*p_{A}(0|0)*p_{C}(0|0) + 0.371*p_{A}(0|0) + 0.228*p_{BC}(00|00)**2 + 0.042*p_{BC}(00|00)*p_{B}(0|0) + 0.044*p_{BC}(00|00)*p_{C}(0|0) + 0.129*p_{BC}(00|00) - 0.443*p_{B}(0|0)**2 - 0.726*p_{B}(0|0)*p_{C}(0|0) + 0.365*p_{B}(0|0) - 0.446*p_{C}(0|0)**2 + 0.369*p_{C}(0|0) + 0.481

In the above, lower indices indicate marginals. For example, $p_{AC}(ac|xz) := \sum_b p(abc|xyz)$. Note that due to no-signaling, in this example the marginal is independent of the setting $y$.

 Finally, given that we only have two outcomes, we can also express the certificate in "correlator form", where the correlators are defined as 
 
 $$\left\langle A_{x} \right\rangle =\sum_{a\in \{0,1\}} (-1)^{a} \, p_{A}(a|x)$$ 
 $$\left\langle A_{x} B_{y} \right\rangle =\sum_{a, b \in \{0,1\}} (-1)^{a+b} \, p_{AB}(ab|xy)$$
 $$\left\langle A_{x} B_{y} C_{z}\right\rangle =\sum_{a, b, c \in \{0,1\}} (-1)^{a+b+c} \, p(abc|xyz)$$ 
 
 where the omitted 2-body and 1-body correlators have similar definitions:

In [6]:
sdprelax.certificate_as_correlators(clean=True)

0.02875*\langle A_{0} B_{0} C_{0} \rangle*\langle A_{0} \rangle + 0.0285*\langle A_{0} B_{0} C_{0} \rangle*\langle B_{0} \rangle + 0.0285625*\langle A_{0} B_{0} C_{0} \rangle*\langle C_{0} \rangle - 0.1628125*\langle A_{0} B_{0} C_{0} \rangle + 0.0145*\langle A_{0} B_{0} \rangle**2 + 0.0064375*\langle A_{0} B_{0} \rangle*\langle A_{0} C_{0} \rangle - 0.0699375*\langle A_{0} B_{0} \rangle*\langle A_{0} \rangle + 0.006*\langle A_{0} B_{0} \rangle*\langle B_{0} C_{0} \rangle - 0.068625*\langle A_{0} B_{0} \rangle*\langle B_{0} \rangle + 0.004625*\langle A_{0} B_{0} \rangle*\langle C_{0} \rangle + 0.20225*\langle A_{0} B_{0} \rangle + 0.0144375*\langle A_{0} C_{0} \rangle**2 - 0.0698125*\langle A_{0} C_{0} \rangle*\langle A_{0} \rangle + 0.0059375*\langle A_{0} C_{0} \rangle*\langle B_{0} C_{0} \rangle + 0.0045*\langle A_{0} C_{0} \rangle*\langle B_{0} \rangle - 0.069*\langle A_{0} C_{0} \rangle*\langle C_{0} \rangle + 0.2025625*\langle A_{0} C_{0} \rangle - 0.037875*\langle A_{0} \rangle*

### Example 2: Critical visibility of the 2PR distribution in the quantum tripartite-line scenario

It is known that the 2PR distribution, defined as:

$$ P_{\text{2PR}}(abc|xyz) := \frac{1+ (-1)^{a+b+c+xy+yz}}{8} $$

is incompatible with the tripartite-line scenario (also called "quantum bilocal scenario"), whose DAG is depicted in the following figure:

<center> <img src="./figures/bilocal_1.PNG" alt="drawing" width="400"/> </center>

This can be shown by running a feasibility program, as in Example 1. We might also be interested in studying how much noise this distribution can tolerate before the relaxation no longer identifies the distribution as incompatible. One simple model of noise is that of a probabilistic mixture with the uniform distribution:

$$ P_{\text{2PR,v}} := v P_{\text{2PR}}  + (1-v)/8 $$

A simple approach would be to vary the parameter $v$ from $v{=}1$ to $v{=}0$ and find the $v_{\text{crit}}$ for which the problem status changes from infeasible to feasible. However, there is a more robust method available.

#### Feasibility as an optimisation

A more numerically robust approach is to convert feasibility problems to optimisation problems. Instead of imposing that the moment matrix $\Gamma$ of the SDP relaxation is posivite semidefinite, we can maximize the minimum eigenvalue of $\Gamma$ and check its sign. Clearly, if the result of the optimisation is negative, then one cannot find a matrix $\Gamma$ that is positive semidefinite, thus the original program is infeasible. 

By setting the flag `feas_as_optim` to `True` in the `InflationSDP.solve()` method, feasibility problems are converted to optimisation problems. The result is stored in `InflationSDP.objective_value`. 

We encode the inflation scenario and generate the relaxation corresponding to NPA level 2:

In [8]:
qbilocal = InflationProblem(dag={"rho_AB": ["A", "B"],
                                 "rho_BC": ["B", "C"]},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[2, 2, 2],
                            inflation_level_per_source=[2, 2])
qbilocal_relax = InflationSDP(qbilocal)
qbilocal_relax.generate_relaxation('npa2')

And next we run a simple bisection to find the $v_{\text{crit}}$ for which the maximum minimum eigenvalue is 0. The bisection is implemented in an auxiliary file:

In [9]:
sdp, v_crit = bisection(qbilocal_relax, P_2PR_array, verbose=0)
v_crit

It is known that $P_{\text{2PR},v}$ is incompatible with the quantum tripartite scenario for $v>\frac{1}{2}$. Quantum inflation at the second order correctly recovers this bound.

## Optimization of Bell operators

One can use inflation techniques to not only run causal compatibility problems, but also to optimize over the generated relaxation, and therefore get upper bounds on the values of various Bell operators.

### Example 3. Upper bounds on Mermin's inequality

Let us consider Mermin's inequality, written in the correlator form introduced in Example 1:

$$ \text{Mermin} = \langle A_1 B_0 C_0 \rangle +  \langle A_0 B_1 C_0 \rangle +  \langle A_0 B_0 C_1 \rangle -  \langle A_1 B_1 C_1 \rangle $$

It is known that the algebraic maximum of 4 is achieved in the tripartite scenario both with global shared randomness and also global non-signaling sources. However, one can see a difference between quantum and general no-signaling sources when restricting to the triangle scenario from Example 1.

First we generate the relaxation corresponding to a second order inflation of the triangle of NPA level 2: 

In [ ]:
qtriangle = InflationProblem(dag={"rho_AB": ["A", "B"],
                                  "rho_BC": ["B", "C"],
                                  "rho_AC": ["A", "C"]}, 
                             outcomes_per_party=[2, 2, 2],
                             settings_per_party=[2, 2, 2],
                             inflation_level_per_source=[2, 2, 2])
sdprelax = InflationSDP(qtriangle)
sdprelax.generate_relaxation('npa2')

We implement the objective function after extracting the measurement operators and solve the program:

In [ ]:
mmnts = sdprelax.measurements
A0, B0, C0, A1, B1, C1 = (1-2*mmnts[party][0][setting][0] for setting in range(2) for party in range(3))

sdprelax.set_objective(objective = A1*B0*C0 + A0*B1*C0 + A0*B0*C1 - A1*B1*C1)
sdprelax.solve()

sdprelax.objective_value

Notice that we get a value that is within numerical precision the algebraic maximum of 4. To improve on this result, we will need to do a tighter SDP relaxation.

#### Customising the generating set for the semidefinite relaxation

To get a tighter SDP relaxation, we will add more monomoials to the generating set. Namely, we will use the union of the monomoials corresponding to NPA level 2 and local level 1.

The so-called "local levels" are a different choice of generating set for the moment matrix. Whereas NPA level $n$ is the $n$-times cartesian product (without duplicated elements) of the set of measurements of the parties together with the identity, local level $n$ refers to a generating set with all the products up to $n$ operators per party. For more details, see [Physical Review X 11.2 (2021): 021043](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.11.021043).

In what follows, we use the built-in method `build_columns` to generate the columns corresponding to NPA level 2 and local level 1. Then we do a union, generate the relaxation and again, solve the program. As it will now take a bit longer, we increase the verbosity level to see the progress: 

In [ ]:
npa2   = sdprelax.build_columns('npa2')
local1 = sdprelax.build_columns('local1')

generatingset = npa2
for mon in local1:
     if mon not in generatingset:
         generatingset.append(mon)

sdprelax.generate_relaxation(generatingset)
sdprelax.set_objective(objective = A1*B0*C0 + A0*B1*C0 + A0*B0*C1 - A1*B1*C1)

sdprelax.verbose = 1
sdprelax.solve()
sdprelax.objective_value

With quantum inflation we can certify then that the Mermin inequality cannot have a value larger than $3.085$ for the quantum triangle causal scenario. 

## Optimisation over classical distributions and feasibility problems 

With quantum inflation, we can also optimize over the relaxation of classical distributions. This works by imposing at the level of the SDP relaxation the constraint that all operators defining the moments commute. The effect of this constraint is that previously different variables in the moment matrix become identical. For example, $\langle A_{x} A_{x'} \rangle \neq \langle A_{x'} A_{x} \rangle$ in general in quantum mechanics, but if we assume all operators commute, then they become equal. 

To enable this feauture one simply adds the flag `commuting=True` when instantiating the `InflationSDP` object. 

### Example 4: Critical visibility of the 2PR distribution in the *classical* tripartite-line scenario

As an example, we find the critical visibility of the $P_{\text{2PR}}$ distribution from Example 2, but in the classical tripartite line scenario with a second order inflation, and with the local level 1 generating set for the SDP relaxation:

In [ ]:
qbilocal = InflationProblem(dag={"rho_AB": ["A", "B"], "rho_BC": ["B", "C"]},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[2, 2, 2],
                            inflation_level_per_source=[2, 2])
qbilocal_relax = InflationSDP(qbilocal, commuting=True)
qbilocal_relax.generate_relaxation('local1')
sdp, v_crit = bisection(qbilocal_relax, P_2PR_array, verbose=0)
v_crit

This relaxation of the set of distributions classically simulable in the tripartite line scenario certifies then the incompatibility of the $P_{\text{2PR}}$ distribution for $v>0.3536$. This does not completely certify incompatibility down to the known critical threshold of $v_{\text{crit}}=\frac{1}{4}$, but we expect tighter relaxations, which are computationally more expensive, might recover this value. 

For optimisation problems, one can run the exact same program as in Example 3, but with the flag `commuting` set to `True`.

## Standard NPA.

If the DAG correspongs to a single global shared source scenario, then doing an inflation does not grant any advantage. In this case, the semidefinite programming relaxation defaults to being the same as the [NPA hierarcy](https://iopscience.iop.org/article/10.1088/1367-2630/10/7/073013). If we set the `commuting` flag to `True` then this is a relaxation of the set of distributions classically with global shared randomness, as introduced in [Phys. Rev. X 7, 021042](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.7.021042). We will show this with two simple examples.

### Example 5: Critical visibility of the PR box in the standard Bell scenario with quantum sources

We recover the critical visibility of $v_{\text{crit}}=\frac{1}{\sqrt{2}}$ for a [Popeschu-Rohrlich box](https://link.springer.com/article/10.1007/BF02058098) in the Bell scenario.  

In [ ]:
bellscenario = InflationProblem(dag={"rho_AB": ["A", "B"]},
                                outcomes_per_party=[2, 2], 
                                settings_per_party=[2, 2])
bellscenario_relax = InflationSDP(bellscenario)
bellscenario_relax.generate_relaxation('npa1')
sdp, v_crit = bisection(bellscenario_relax, P_PRbox_array, verbose=0)
print("Critical visibility: ", v_crit)
sdp.certificate_as_correlators(clean=True)

Notice that the dual certificate that we extract in correlator form is exactly the CHSH inequality tangent to the quantum set of correlations.

### Example 6: Optimising the CHSH inequality in the Bell scenario

Now we will showcase optimising the CHSH inequality that was recovered in Example 5:

In [ ]:
bellscenario_relax = InflationSDP(bellscenario, commuting=True)
bellscenario_relax.generate_relaxation('npa1')

mmnts = bellscenario_relax.measurements
A0, B0, A1, B1 = (1-2*mmnts[party][0][setting][0] for setting in range(2) for party in range(2))
CHSH = A0*B0 + A0*B1 + A1*B0 - A1*B1

bellscenario_relax.set_objective(CHSH)
bellscenario_relax.solve()
print("CHSH upper bound in the Bell scenario with a quantum source: ", bellscenario_relax.objective_value)

We correctly recover the Tsirelson bound for the CHSH inequality.